# 1번

In [1]:
import numpy as np

In [2]:
A = np.array([[2,0,2,1,3,2],
              [0,2,0,2,2,2],
              [1,0,1,3,1,1],
              [0,0,1,1,1,0],
              [0,1,3,4,1,0],
              [0,1,0,0,5,2]])
A.shape

(6, 6)

* 3 x 3 대각선 filter 적용
size =( 6 - 3 + 2 x 1)/1 + 1 => 6

In [3]:
A_zero = np.array([[0,0,0,0,0,0,0,0],
               [0,2,0,2,1,3,2,0],
               [0,0,2,0,2,2,2,0],
               [0,1,0,1,3,1,1,0],
               [0,0,0,1,1,1,0,0],
               [0,0,1,3,4,1,0,0],
               [0,0,1,0,0,5,2,0],
               [0,0,0,0,0,0,0,0]])
A_zero

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 2, 1, 3, 2, 0],
       [0, 0, 2, 0, 2, 2, 2, 0],
       [0, 1, 0, 1, 3, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 1, 3, 4, 1, 0, 0],
       [0, 0, 1, 0, 0, 5, 2, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [4]:
A_ = np.array([[4,0,4,3,5,2],
               [0,5,3,5,4,5],
               [1,1,4,4,3,3],
               [1,4,5,3,4,1],
               [1,1,3,10,4,1],
               [0,1,1,3,9,3]])
print(A_.shape)
A_

(6, 6)


array([[ 4,  0,  4,  3,  5,  2],
       [ 0,  5,  3,  5,  4,  5],
       [ 1,  1,  4,  4,  3,  3],
       [ 1,  4,  5,  3,  4,  1],
       [ 1,  1,  3, 10,  4,  1],
       [ 0,  1,  1,  3,  9,  3]])

* 2 x 2 maxpooling 적용
size -> 3 x 3

In [5]:
A_ = np.array([[5,5,5],
               [4,5,4],
               [1,10,9]])
print(A.shape)
A_

(6, 6)


array([[ 5,  5,  5],
       [ 4,  5,  4],
       [ 1, 10,  9]])

# 2번

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# MNIST datasets
mnist_train = dsets.MNIST(root='MNIST_data/',
                         train = True,
                         transform = transforms.ToTensor(),
                         download = True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

## 숙제 모델

In [11]:
class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size = 4, stride = 1, padding = 1),
            # ? x 32 x 27 x 27
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
            # ? x 32 x 13 x 13
        
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            # ? x 64 x 13 x 13
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
            # ? x 64 x 6 x 6
        
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size = 3, stride = 2, padding = 1),
            # ? x 128 x 3 x 3
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1))
            # ? x 128 x 2 x 2
        
        self.fc1 = torch.nn.Linear(128 * 2 * 2, 625, bias = True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p = 1 - self.keep_prob))
        
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out

### layer3까지 지난 image의 output shape이 2 x 2 가 됨

In [12]:
model = CNN().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
total_batch = len(data_loader)
model.train()
print('Learning started. It takes sometime')
for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished')

Learning started. It takes sometime
[Epoch:    1] cost = 0.260818541
[Epoch:    2] cost = 0.0653349981
[Epoch:    3] cost = 0.0462695546
[Epoch:    4] cost = 0.0356797874
[Epoch:    5] cost = 0.0290069245
[Epoch:    6] cost = 0.0246454626
[Epoch:    7] cost = 0.0211415365
[Epoch:    8] cost = 0.0169709064
[Epoch:    9] cost = 0.0155284861
[Epoch:   10] cost = 0.0168496426
[Epoch:   11] cost = 0.0121403234
[Epoch:   12] cost = 0.0110005122
[Epoch:   13] cost = 0.0121472739
[Epoch:   14] cost = 0.00828384608
[Epoch:   15] cost = 0.00880440697
Learning Finished


In [14]:
with torch.no_grad():
    model.eval()
    
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

C:\Users\user\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\user\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy:  0.9876999855041504


## 기존 모델

In [6]:
# CNN Model
class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        # L! image shape = (?, 28, 28, 1)
        #    Conv       -> (?, 28, 28, 32)
        #    Pool       -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
        # L2 image shape = (?, 14, 14, 32)
        #    Conv       -> (?, 14 ,14 ,64)
        #    Pool       -> (?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
        # L3 image shape = (?, 7, 7, 64)
        #    Conv       -> (?, 7, 7, 128)
        #    Pool       -> (?, 4, 4, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1))
        
        # L4 FC 4 x 4 x 128 inputs - > 625 outputs
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias = True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p = 1 - self.keep_prob))
        
        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = torch.nn.Linear(625, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [7]:
model = CNN().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
total_batch = len(data_loader)
model.train()   # set the model to train mode (dropout = True)
print('Learning started. It takes sometime')
for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished')

Learning started. It takes sometime
[Epoch:    1] cost = 0.190713644
[Epoch:    2] cost = 0.0520608276
[Epoch:    3] cost = 0.0376487821
[Epoch:    4] cost = 0.0285026077
[Epoch:    5] cost = 0.0243023224
[Epoch:    6] cost = 0.0195404254
[Epoch:    7] cost = 0.0160752013
[Epoch:    8] cost = 0.0142789194
[Epoch:    9] cost = 0.0117072025
[Epoch:   10] cost = 0.0123375095
[Epoch:   11] cost = 0.0106206443
[Epoch:   12] cost = 0.0107046869
[Epoch:   13] cost = 0.00735590933
[Epoch:   14] cost = 0.00858276151
[Epoch:   15] cost = 0.00802615285
Learning Finished


In [9]:
with torch.no_grad():
    model.eval()   # set the model to evaluation mode (dropout = False)
    
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

C:\Users\user\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\user\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy:  0.9864000082015991


### 결론
 * 기존 것보다 최종 Loss는 조금 높지만 accuracy는 더 좋게 나옴 => 근데 거의 차이없는듯